In [1]:
import numpy as np
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
import os

from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [2]:
from tqdm import tqdm
from utils import *

In [5]:
class ObjectDetectionDataset(Dataset):
    '''
    A Pytorch Dataset class to load the images and their corresponding annotations.
    
    Returns
    ------------
    images: torch.Tensor of size (B, C, H, W)
    gt bboxes: torch.Tensor of size (B, max_objects, 4)
    gt classes: torch.Tensor of size (B, max_objects)
    '''
    def __init__(self, csv_path, img_size, name2idx, img_dir = None):
        self.annotation_path = csv_path
        self.img_size = img_size
        self.name2idx = name2idx
        
        self.img_data_all, self.gt_bboxes_all, self.gt_classes_all = self.get_data()
        
    def __len__(self):
        return self.img_data_all.size(dim=0)
    
    def __getitem__(self, idx):
        return self.img_data_all[idx], self.gt_bboxes_all[idx], self.gt_classes_all[idx]
        
    def get_data(self):
        img_data_all = []
        gt_idxs_all = []
        
        if img_dir is None:
            gt_boxes_all, gt_classes_all, img_paths = parse_annotation(self.annotation_path, self.img_size)
        else:
            gt_boxes_all, gt_classes_all, img_paths = parse_annotation(self.annotation_path, self.img_size, img_dir = img_dir)
        
        for i, img_path in tqdm(enumerate(img_paths), total=len(img_paths)):
      
            # skip if the image path is not valid
            if (not img_path) or (not os.path.exists(img_path)):
                continue
            
            # read and resize image
            
            img = io.imread(img_path)
            img = resize(img, self.img_size)
            
            # convert image to torch tensor and reshape it so channels come first
            img_tensor = torch.from_numpy(img).permute(2, 0, 1)
            
            # encode class names as integers
            gt_classes = gt_classes_all[i]
            gt_idx = torch.Tensor([self.name2idx[name] for name in gt_classes])
            
            img_data_all.append(img_tensor)
            gt_idxs_all.append(gt_idx)
        
        # pad bounding boxes and classes so they are of the same size

        if len(gt_boxes_all)!=0 and len(gt_idxs_all)!=0 :
          
          gt_bboxes_pad = pad_sequence(gt_boxes_all, batch_first=True, padding_value=-1)
          gt_classes_pad = pad_sequence(gt_idxs_all, batch_first=True, padding_value=-1)
        
        # stack all images
        print(img_data_all)
        img_data_stacked = torch.stack(img_data_all)[:, :3, :, :]
        
        return img_data_stacked.to(dtype=torch.float32), gt_bboxes_pad, gt_classes_pad

In [4]:
img_width = 640
img_height = 480
csv_path = "/content/APS360_Traffic_Sign_Recognition/dataset_traffic_signs.csv"
image_dir = os.path.join("data", "images")
name2idx = {'pad': -1, '30kmh': 0,'60kmh':1, '100kmh' : 2, 'yield': 3, 'keepRight' :4, 'NoEntry':5, 'NoLeft': 6, 'Stop':7, 'noRight':8, 'ChildrenCrossing' :9 }
idx2name = {v:k for k, v in name2idx.items()}

In [9]:
train_csv_path = "C:/Users/gabri/Documents/UofT/APS360/Project/Code2/APS360_Traffic_Sign_Recognition/train.csv"
val_csv_path = "C:/Users/gabri/Documents/UofT/APS360/Project/Code2/APS360_Traffic_Sign_Recognition/val.csv"

In [6]:
img_dir = "C:/Users/gabri/Documents/UofT/APS360/Project/Code2/APS360_Traffic_Sign_Recognition/"

In [8]:
od_dataset = ObjectDetectionDataset(train_csv_path, (img_height, img_width), name2idx)

 57%|█████▋    | 1282/2243 [06:29<04:52,  3.29it/s]  


MemoryError: Unable to allocate 9.38 MiB for an array with shape (480, 640, 4) and data type float64

In [10]:
df_val = pd.read_csv(val_csv_path)

In [11]:
len(df_val)

922